# 问题1

编写一个程序, ```get_response(saying, response_rules)```输入是一个字符串 + 我们定义的 rules，例如上边我们所写的 pattern， 输出是一个回答。

In [1]:
import random 
import jieba
from collections import defaultdict

In [2]:
"""
判断匹配规则中的  ?X
"""
def is_variable(pat):
        return pat.startswith('?') and all(s.isalpha() for s in pat[1:])

In [3]:
"""
递归逐分字判断匹配规则与输入语句
"""

def pat_match(pattern, saying):
        if not pattern or not saying: return []

        if is_variable(pattern[0]):
            return [(pattern[0], saying[0])] + pat_match(pattern[1:], saying[1:])
        else:
            if pattern[0] != saying[0]:
                return []
            else:
                return pat_match(pattern[1:], saying[1:])

In [4]:
"""
将查找出的匹配分词转为字典
"""

def pat_to_dict(patterns):
        return {k: v for k, v in patterns}

In [5]:
"""
根据查找出的匹配分词对回答中的?X进行替换
"""
def subsitite(rule, parsed_rules):
        if not rule: return []
        return [parsed_rules.get(rule[0], rule[0])] +subsitite(rule[1:], parsed_rules)

In [6]:
"""
先用遍历查找是否有与输入语句匹配的规则，然后再根据查找到的规则进行回答替换。
"""

def get_response(saying,rules):

    for rule,response in rules.items():
        john_pat=pat_match(rule.split(),saying.split())
        if john_pat!=[]:
            return ' '.join(subsitite(random.choice(response).split(),pat_to_dict(john_pat)))

    if not john_pat:
        return 'I am sorry for not understanding your word. Please try again.'

    return ''

In [7]:
defined_patterns = {
    "I need ?X": ["Image you will get ?X soon", "Why do you need ?X ?"], 
    "My ?X told me something": ["Talk about more about your ?X", "How do you think about your ?X ?"]
    }
print(get_response('I need xps9380',defined_patterns))


Image you will get xps9380 soon


#上面只能匹配单字无空格的情况，我们接下来将其修改为可以匹配多个字符。

In [8]:
"""
修改pat_to_dict函数，将字典的值变为带空格的字符串。
"""

def pat_to_dict(patterns):
    return {k: ' '.join(v) if isinstance(v, list) else v for k, v in patterns}

In [9]:
"""
新的匹配函数， 匹配?*关键字。
"""
def is_pattern_segment(pattern):
    return pattern.startswith('?*') and all(a.isalpha() for a in pattern[2:])

In [10]:
"""
将该函数将逐分词匹配规则与输入语句，并生成关键字替换字典。
其中加入了边界条件的判断以防止错误回复。
边界情况主要为：输入语句消耗完了，但匹配规则依然有剩余的分词，此时应返回fail，而不是字典里的内容。

"""
fail = [True, None]

def pat_match_with_seg(pattern, saying):

    if not pattern or not saying: return []

    pat = pattern[0]

    if is_variable(pat):
        if pattern[1:] and not saying[1:]: 
            return fail
        else:
            return [(pat, saying[0])] + pat_match_with_seg(pattern[1:], saying[1:])
    elif is_pattern_segment(pat):
        match, index = segment_match(pattern, saying)
        if pattern[1:] and not saying[index:]:
            return fail

        return [match] + pat_match_with_seg(pattern[1:], saying[index:])
    elif pat == saying[0]:
        if pattern[1:] and not  saying[1:]: 
            return fail
        else:
            return pat_match_with_seg(pattern[1:], saying[1:])
    else:
        return fail

In [11]:
"""
用于一次性匹配多个分词，并将?*转为?
返回结果为以匹配的字典值，与saying继续改匹配的地方
"""
def segment_match(pattern, saying):
    seg_pat, rest = pattern[0], pattern[1:]
    seg_pat = seg_pat.replace('?*', '?')

    if not rest: return (seg_pat, saying), len(saying)    

    for i, token in enumerate(saying):
        if rest[0] == token:  
            return (seg_pat, saying[:i]), i

    return (seg_pat, saying), len(saying)

In [12]:
"""
依然是将输入与规则进行遍历查询。
其中会有开头的?*X将语句消耗完，但是返回字典中带fail的情况，遇到该情况直接将结果重置为fail。
"""
def get_response(saying,rules):
        
    for rule,response in rules.items():
        john_pat=pat_match_with_seg(rule.split(),saying.split())
        for j in john_pat:
            if j==None:
                john_pat=fail
                continue
        if john_pat!=fail:
            return ' '.join(subsitite(random.choice(response).split(),pat_to_dict(john_pat)))

    return 'I am sorry for not understanding your word. Please try again.'

In [13]:
rule_responses = {
    'I need ?*X':['Image you will get ?X soon', 'Why do you need ?X ?'], 
    '?*X I feel ?*Y': ['Do you often feel ?Y ?', 'What other feelings do you have?'],
    '?*P is very good and ?*X':['May be you can show me!','What a coincidence！ ?P is very good and ?X ,too!'],
    '?*x': ['I am sorry for not understanding your word. Please try again.']
}

print(get_response('I need Dell XPS 9380',rule_responses))
print(get_response('Ewww I feel terrible',rule_responses))
print(get_response('My dog is very good and my cat is very cute',rule_responses))
print(get_response('I',rule_responses))

Why do you need Dell XPS 9380 ?
Do you often feel terrible ?
May be you can show me!
I am sorry for not understanding your word. Please try again.


# 问题2

之后将其修改为可支持中文输入与规则的情况。

In [14]:
"""
lcut_processor 用来处理所有的进入的字符串，不管中英。
代替前面的split()函数。
其中要注意结尾的?的匹配。
将?X与?*X重新组合成一个列表元素，并沿用上面写好了的回答模块。

"""
def lcut_processor(seg_list):
    seg_list=jieba.lcut(seg_list)
    new_list=[]
    i=0
    while i<len(seg_list):
        if seg_list[i]=='?'and i!=(len(seg_list)-1):
            if seg_list[i+1]=='*':
                if seg_list[i+2].isalpha():
                    new_list.append('?*'+(seg_list[i+2])) 
                    i+=3
            elif seg_list[i+1].isalpha():
                new_list.append('?'+(seg_list[i+1]))
                i+=2
            else:
                new_list.append(seg_list[i])
                i+=1
        elif seg_list[i]==' ':
                i+=1
        else:
            new_list.append(seg_list[i])
            i+=1
    return new_list

In [15]:
"""
用来判断回答中是否有中文，若是有，就不需要用空格分割词语。
若是没有，则为英文句子，就需要用空格分割。
"""

def is_Chinese(sentence):
    for word in sentence:
        for ch in word:
            if '\u4e00' <= ch <= '\u9fff':
                return True
    return False

In [16]:
"""
思路与之前相似，先是遍历规则，然后随机挑选回答并判断中英文，给出回复。
"""
def get_response(saying,rules):
        
    for rule,response in rules.items():
        john_pat=pat_match_with_seg(lcut_processor(rule),lcut_processor(saying)) 
        for j in john_pat:
            if j==None:
                john_pat=fail
                continue
        if john_pat!=fail:
            Reply=subsitite(lcut_processor(random.choice(response)),pat_to_dict(john_pat))
            if is_Chinese(Reply):
                return ''.join(Reply)
            else:
                return ' '.join(Reply)
    return 'I am sorry for not understanding your word. Please try again.'

In [17]:
rule_responses = {
    'I need ?*X':['Image you will get ?X soon', 'Why do you need ?X ?'], 
    '?*X I feel ?*Y': ['Do you often feel ?Y ?', 'What other feelings do you have?'],
    '?*P is very good and ?*X':['May be you can show me!','What a coincidence！ ?P is very good and ?X ,too!'],
    '?*x喜欢?*y': ['喜欢?y的哪里？', '?y有什么好的呢？', '你想要?y吗？'],
    '?*x': ['I am sorry for not understanding your word. Please try again.']
}
print(get_response('我喜欢DELL XPS 9380',rule_responses))
print(get_response('I need XPS 9380',rule_responses))
print(get_response('My dog is very good and my cat is very cute',rule_responses))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\dell\AppData\Local\Temp\jieba.cache
Loading model cost 0.769 seconds.
Prefix dict has been built succesfully.


喜欢DELL XPS 9380的哪里？
Image you will get XPS 9380 soon
What a coincidence ！ My dog is very good and my cat is very cute , too !


# 问题3

历史记录模式，在完成一次对话之后，将模式中匹配到的?X关键词字典储存下来，并添加到模式中，以呈现一种机器人记忆的方式。
如：
    人：我喜欢猫
    机器：我也是。
    人：我喜欢狗
    机器：但你上次说你喜欢猫。
    
在与一个人对话的同时，机器为这个人建立语料库，并储存这些历史信息。

# 问题4

1. 这样的程序有什么优点？有什么缺点？你有什么可以改进的方法吗？ 

    答：优点是，程序应答速度快，在面对一些“有规律的、反复的”问答时，一旦匹配到了特定的规则，机器人可以快速的完成问答任务。
        缺点是，规则是死的，出现规则外的问答时，直接失去作用。其次规则的增改删查依然需要人类来进行操作。
        
        我个人认为可以通过语料库的方式储存对话，在面对同一个人的问答时，机器人可以搜索历史记录来“获得对话历史信息”以增加对话的丰富程度。在对话失败的记录("I don't understand you")中，可以用聚类的方式向规则库增加新的规则。
    



2. 什么是数据驱动？数据驱动在这个程序里如何体现？

   答：个人的理解是，新的数据能够影响机器进行决策，就已经是数据驱动了。再往深了说，机器能够从数据中获得“信息”，并根据这个信息来进行决策，也是数据驱动的一种体现。
   
       在这个程序中，我们的输入语句就是数据，机器处理了这份数据(匹配规则，分割语句等)，并作出不同的决策(回答)，就是体现了数据驱动的思想。


3. 数据驱动与 AI 的关系是什么？ 
    
    答：AI的学习能力，来自于数据驱动。
        我们的程序设计让AI程序有了能够处理并提取信息的能力，然后AI不断的处理数据，并从中提取信息，进行学习，这一切的根源就是数据。
        没有新新新的数据，AI的处理提取能力再强大，也无法继续学习到新的知识，准确率不再上升。
    
        